# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 2:34PM,237679,10,MET210383,"Methapharm, Inc.",Released
1,May 19 2022 2:34PM,237679,10,MET210384,"Methapharm, Inc.",Released
2,May 19 2022 2:34PM,237679,10,MET210385,"Methapharm, Inc.",Released
3,May 19 2022 2:34PM,237679,10,MET210386,"Methapharm, Inc.",Released
4,May 19 2022 2:34PM,237679,10,MET210387,"Methapharm, Inc.",Released
5,May 19 2022 2:34PM,237679,10,MET210390,"Methapharm, Inc.",Released
6,May 19 2022 2:34PM,237679,10,MET210391,"Methapharm, Inc.",Released
7,May 19 2022 2:34PM,237679,10,MET210392,"Methapharm, Inc.",Released
8,May 19 2022 2:34PM,237679,10,MET210393,"Methapharm, Inc.",Released
9,May 19 2022 2:34PM,237679,10,MET210394,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,237672,Released,3
45,237676,Released,1
46,237678,Released,4
47,237679,Released,19
48,237680,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237672,NaN,5.0,3.0
237676,NaN,NaN,1.0
237678,NaN,NaN,4.0
237679,NaN,NaN,19.0
237680,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,0.0,0.0,5.0
237571,0.0,1.0,0.0
237575,0.0,1.0,0.0
237583,0.0,3.0,22.0
237586,0.0,0.0,22.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,0,0,5
237571,0,1,0
237575,0,1,0
237583,0,3,22
237586,0,0,22


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,0,0,5
1,237571,0,1,0
2,237575,0,1,0
3,237583,0,3,22
4,237586,0,0,22


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,,,5
1,237571,,1,
2,237575,,1,
3,237583,,3,22
4,237586,,,22


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 2:34PM,237679,10,"Methapharm, Inc."
19,May 19 2022 2:34PM,237680,10,"Methapharm, Inc."
27,May 19 2022 2:31PM,237678,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
31,May 19 2022 2:24PM,237676,12,Twinlab Consolidated Corporation
32,May 19 2022 1:57PM,237560,20,"ACI Healthcare USA, Inc."
37,May 19 2022 1:43PM,237672,10,ISDIN Corporation
45,May 19 2022 1:38PM,237670,10,ISDIN Corporation
59,May 19 2022 1:27PM,237666,16,"SHL Pharma, LLC"
60,May 19 2022 1:23PM,237663,10,Bio-PRF
78,May 19 2022 1:18PM,237661,16,"SHL Pharma, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 19 2022 2:34PM,237679,10,"Methapharm, Inc.",,,19
1,May 19 2022 2:34PM,237680,10,"Methapharm, Inc.",,,8
2,May 19 2022 2:31PM,237678,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,4
3,May 19 2022 2:24PM,237676,12,Twinlab Consolidated Corporation,,,1
4,May 19 2022 1:57PM,237560,20,"ACI Healthcare USA, Inc.",,,5
5,May 19 2022 1:43PM,237672,10,ISDIN Corporation,,5,3
6,May 19 2022 1:38PM,237670,10,ISDIN Corporation,,,14
7,May 19 2022 1:27PM,237666,16,"SHL Pharma, LLC",,,1
8,May 19 2022 1:23PM,237663,10,Bio-PRF,,,18
9,May 19 2022 1:18PM,237661,16,"SHL Pharma, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 2:34PM,237679,10,"Methapharm, Inc.",19,,
1,May 19 2022 2:34PM,237680,10,"Methapharm, Inc.",8,,
2,May 19 2022 2:31PM,237678,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4,,
3,May 19 2022 2:24PM,237676,12,Twinlab Consolidated Corporation,1,,
4,May 19 2022 1:57PM,237560,20,"ACI Healthcare USA, Inc.",5,,
5,May 19 2022 1:43PM,237672,10,ISDIN Corporation,3,5,
6,May 19 2022 1:38PM,237670,10,ISDIN Corporation,14,,
7,May 19 2022 1:27PM,237666,16,"SHL Pharma, LLC",1,,
8,May 19 2022 1:23PM,237663,10,Bio-PRF,18,,
9,May 19 2022 1:18PM,237661,16,"SHL Pharma, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 2:34PM,237679,10,"Methapharm, Inc.",19,,
1,May 19 2022 2:34PM,237680,10,"Methapharm, Inc.",8,,
2,May 19 2022 2:31PM,237678,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4,,
3,May 19 2022 2:24PM,237676,12,Twinlab Consolidated Corporation,1,,
4,May 19 2022 1:57PM,237560,20,"ACI Healthcare USA, Inc.",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 2:34PM,237679,10,"Methapharm, Inc.",19.0,NaN,NaN
1,May 19 2022 2:34PM,237680,10,"Methapharm, Inc.",8.0,NaN,NaN
2,May 19 2022 2:31PM,237678,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4.0,NaN,NaN
3,May 19 2022 2:24PM,237676,12,Twinlab Consolidated Corporation,1.0,NaN,NaN
4,May 19 2022 1:57PM,237560,20,"ACI Healthcare USA, Inc.",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 2:34PM,237679,10,"Methapharm, Inc.",19.0,0.0,0.0
1,May 19 2022 2:34PM,237680,10,"Methapharm, Inc.",8.0,0.0,0.0
2,May 19 2022 2:31PM,237678,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4.0,0.0,0.0
3,May 19 2022 2:24PM,237676,12,Twinlab Consolidated Corporation,1.0,0.0,0.0
4,May 19 2022 1:57PM,237560,20,"ACI Healthcare USA, Inc.",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2851772,168.0,31.0,5.0
12,237676,1.0,0.0,0.0
16,950530,4.0,0.0,0.0
17,237654,1.0,0.0,0.0
18,1188024,5.0,0.0,0.0
19,1425910,12.0,10.0,30.0
20,1425700,72.0,13.0,0.0
21,237575,0.0,1.0,0.0
22,950491,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2851772,168.0,31.0,5.0
1,12,237676,1.0,0.0,0.0
2,16,950530,4.0,0.0,0.0
3,17,237654,1.0,0.0,0.0
4,18,1188024,5.0,0.0,0.0
5,19,1425910,12.0,10.0,30.0
6,20,1425700,72.0,13.0,0.0
7,21,237575,0.0,1.0,0.0
8,22,950491,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,168.0,31.0,5.0
1,12,1.0,0.0,0.0
2,16,4.0,0.0,0.0
3,17,1.0,0.0,0.0
4,18,5.0,0.0,0.0
5,19,12.0,10.0,30.0
6,20,72.0,13.0,0.0
7,21,0.0,1.0,0.0
8,22,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,168.0
1,12,Released,1.0
2,16,Released,4.0
3,17,Released,1.0
4,18,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21,22
Status,,,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0
Executing,31.0,0.0,0.0,0.0,0.0,10.0,13.0,1.0,1.0
Released,168.0,1.0,4.0,1.0,5.0,12.0,72.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0
1,Executing,31.0,0.0,0.0,0.0,0.0,10.0,13.0,1.0,1.0
2,Released,168.0,1.0,4.0,1.0,5.0,12.0,72.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0
1,Executing,31.0,0.0,0.0,0.0,0.0,10.0,13.0,1.0,1.0
2,Released,168.0,1.0,4.0,1.0,5.0,12.0,72.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()